In [41]:
# Import Dependencies
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from IPython.display import display, Image
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50, DenseNet121, EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
import os
from keras.utils import to_categorical

In [42]:
# Set up Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
# Change Directory
%cd /content/drive/MyDrive/pet_expressions_data

/content/drive/MyDrive/pet_expressions_data


In [44]:
# Define the path to the dataset folders
happy_folder = "/content/drive/MyDrive/pet_expressions_data/happy"
sad_folder = "/content/drive/MyDrive/pet_expressions_data/sad"
angry_folder = "/content/drive/MyDrive/pet_expressions_data/angry"
other_folder = "/content/drive/MyDrive/pet_expressions_data/other"

In [45]:
# Check the number of pictures in each 'emotion' folder, ideally we would want this to be equal
# Define function to count picturues within a folder, takes path as input

def count_pics(folder_path):
    picture_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    picture_count = 0

    # Check if the provided folder path exists
    if not os.path.exists(folder_path):
        print("Folder path does not exist.")
        return 0

    # Get a list of files in the folder
    files = os.listdir(folder_path)

    # Count pictures with valid extensions
    for file in files:
        _, extension = os.path.splitext(file)
        if extension.lower() in picture_extensions:
            picture_count += 1

    return picture_count

In [46]:
# Happy
happy_picture_count = count_pics(happy_folder)
print(f"Number of pictures in the HAPPY folder: {happy_picture_count}")

Number of pictures in the HAPPY folder: 312


In [47]:
# Sad
sad_picture_count = count_pics(sad_folder)
print(f"Number of pictures in the SAD folder: {sad_picture_count}")

Number of pictures in the SAD folder: 305


In [48]:
# Angry
angry_picture_count = count_pics(angry_folder)
print(f"Number of pictures in the ANGRY folder: {angry_picture_count}")

Number of pictures in the ANGRY folder: 294


In [49]:
 # Other
other_picture_count = count_pics(other_folder)
print(f"Number of pictures in the  OTHER folder: {other_picture_count}")

Number of pictures in the  OTHER folder: 259


In [50]:
# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images

In [51]:
# Load images and labels for each emotion
happy_images = load_images_from_folder(happy_folder)
sad_images = load_images_from_folder(sad_folder)
angry_images = load_images_from_folder(angry_folder)
other_images = load_images_from_folder(other_folder)

In [53]:
# Create labels for each emotion category
happy_labels = [0] * len(happy_images)
sad_labels = [1] * len(sad_images)
angry_labels = [2] * len(angry_images)
other_labels = [3] * len(other_images)

In [54]:
# Concatenate images and labels
x = np.array(happy_images + sad_images + angry_images + other_images)
y = np.array(happy_labels + sad_labels + angry_labels + other_labels)

In [55]:
# Normalize pixel values to range [0, 1]
x = x.astype('float32') / 255.0

In [56]:

# One-hot encode the labels

from tensorflow.keras.utils import to_categorical

y = to_categorical(y, 4)

In [57]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

In [58]:
# Data pre-processing - resizing images to specific shapes for model input
input_shape_densenet = (224, 224, 3)

# Function to resize pictures to variable shapes defined above
def resize_pics(images, input_shape):
    resized_images = []
    for img in images:
        img_resized = cv2.resize(img, (input_shape[0], input_shape[1]))
        img_resized = np.expand_dims(img_resized, axis=-1)
        img_resized = np.repeat(img_resized, 3, axis=-1)  # Add three channels to convert grayscale to RGB
        resized_images.append(img_resized)
    return np.array(resized_images)

# Utilize function above to resize data
x_train_resized_densenet = resize_pics(x_train, input_shape_densenet)


In [59]:
# Fit DenseNet Model
densenet_base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape_densenet)
densenet_base_model.trainable = False

29084464/29084464 [==============================] - 0s 0us/step


In [66]:
# Add custom classification head to the DenseNet model
densenet_global_avg_pooling = GlobalAveragePooling2D()(densenet_base_model.output)
densenet_output = Dense(4, activation='softmax')(densenet_global_avg_pooling)
densenet_model = Model(inputs=densenet_base_model.input, outputs=densenet_output)

In [67]:
# Compile the DenseNet model
densenet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [68]:
# Establish early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-7)

In [69]:
# Passing epochs through variable
######EPOCS ARE ONLY SET TO 5 WARNING: LONG RUN TIME####
epochs=50

In [70]:
# Train the models on resized training data and validation split,monitoring the validation loss and using the early stopping and learning rate scheduling callbacks to stop when necessary.
######EPOCS ARE ONLY SET TO 5 WARNING: LONG RUN TIME####
densenet_history = densenet_model.fit(x_train_resized_densenet, y_train, batch_size=64, epochs=epochs, validation_split=0.1,callbacks=[early_stopping, lr_scheduler])


Epoch 1/50
14/14 [==============================] - 129s 9s/step - loss: 1.5317 - accuracy: 0.2767 - val_loss: 1.4231 - val_accuracy: 0.3191 - lr: 0.0010
Epoch 2/50
14/14 [==============================] - 121s 9s/step - loss: 1.3749 - accuracy: 0.3325 - val_loss: 1.3662 - val_accuracy: 0.3298 - lr: 0.0010
Epoch 3/50
14/14 [==============================] - 121s 9s/step - loss: 1.2947 - accuracy: 0.3943 - val_loss: 1.3324 - val_accuracy: 0.4043 - lr: 0.0010
Epoch 4/50
14/14 [==============================] - 119s 9s/step - loss: 1.2285 - accuracy: 0.4192 - val_loss: 1.3033 - val_accuracy: 0.3830 - lr: 0.0010
Epoch 5/50
14/14 [==============================] - 120s 9s/step - loss: 1.1893 - accuracy: 0.4525 - val_loss: 1.3379 - val_accuracy: 0.3511 - lr: 0.0010
Epoch 6/50
14/14 [==============================] - 120s 9s/step - loss: 1.1730 - accuracy: 0.4679 - val_loss: 1.2508 - val_accuracy: 0.3936 - lr: 0.0010
Epoch 7/50
14/14 [==============================] - 122s 9s/step - loss: 1.1

In [71]:
# Resize test images to the input shape required by each model
x_test_resized_densenet = resize_pics(x_test, input_shape_densenet)

In [72]:
# Evaluate the models on test data
densenet_loss, densenet_accuracy = densenet_model.evaluate(x_test_resized_densenet, y_test)

print("DenseNet Test accuracy:", densenet_accuracy)

8/8 [==============================] - 27s 3s/step - loss: 1.0391 - accuracy: 0.6282
DenseNet Test accuracy: 0.6282051205635071


In [73]:
# Create loss plot
import plotly.graph_objects as go

epochs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

# Create the line plot
fig = go.Figure(data=go.Scatter(x=epochs,y=densenet_history.history['loss'], mode='lines+markers'))

# Update layout
fig.update_layout(title='Loss Plot',
                  xaxis_title='Epochs',
                  yaxis_title='Loss')

# Show the plot
fig.show()

In [74]:
# Create accuracy plot
import plotly.graph_objects as go

# Create the line plot
fig = go.Figure(data=go.Scatter(x=epochs,y=densenet_history.history['accuracy'], mode='lines+markers'))

# Update layout
fig.update_layout(title='Accuracy Plot',
                  xaxis_title='Epochs',
                  yaxis_title='Accuracy')

# Show the plot
fig.show()

In [ ]:
##################################################################################

In [76]:
# Save the trained model
densenet_model.save("pet_expressions_model_DenseNet.h5")

In [ ]:
##################################################################################
#FROM HERE DOWN IS COPIED FROM OUR SEQUENTIAL MODEL NOTEBOOK FILE. IT MAY/MAYNOT WORK FOR DEPLOYING THIS MODEL?

In [ ]:
from keras.models import load_model
# Load the saved model
loaded_model = load_model('pet_expressions_model.h5')

In [ ]:
# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images

In [ ]:
# Load a custom test image
custom_test_image_path = ""

custom_test_image = cv2.imread(custom_test_image_path)
custom_test_image = cv2.cvtColor(custom_test_image, cv2.COLOR_BGR2GRAY)
custom_test_image = cv2.resize(custom_test_image, (48, 48))
custom_test_image = custom_test_image.astype('float32') / 255.0

In [ ]:
# Reshape the image to match the model input shape
custom_test_image = np.expand_dims(custom_test_image, axis=0)
custom_test_image = np.expand_dims(custom_test_image, axis=-1)

In [ ]:
# Make predictions on the custom test image
prediction = loaded_model.predict(custom_test_image)
prediction_prob = prediction[0]

In [ ]:
emotion_label = np.argmax(prediction[0])

In [ ]:
# Map the predicted label to emotion class
emotion_classes = {0: 'happy', 1: 'sad', 2: 'angry', 3: 'feeling some type of way...'}
predicted_emotion = emotion_classes[emotion_label]

In [ ]:
# Print the custom test image and its predicted label
print(f"Predicted Emotion: {predicted_emotion}")
print(f"Confidence [happy, sad, angry,feeling some type of way...]: {prediction_prob}")

In [ ]:
import matplotlib.pyplot as plt

#Display the custom test image using matplotlib
plt.imshow(custom_test_image[0, :, :, 0])
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
from PIL import Image
# Display the original custom test image using PIL
img_pil = Image.open(custom_test_image_path)
plt.imshow(np.array(img_pil))
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()